In [ ]:
from collections import defaultdict
from glob import glob
from pathlib import Path

results_directory = "results-outputs"
stats_directory = "results-stats"

## Results

In [ ]:
diffs = defaultdict(lambda: defaultdict(dict))  # method -> metric -> shift -> value
values = defaultdict(lambda: defaultdict(dict))  # method -> metric -> shift -> value
metrics = [
    "note precision",
    "note recall",
    "note f1",
    "pre-notes frame Precision",
    "pre-notes frame Recall",
    "pre-notes frame F1",
    "post-notes frame Precision",
    "post-notes frame Recall",
    "post-notes frame F1",
]

for file in glob(results_directory + "/*.out"):
    name = Path(file).stem[8:]
    last_index = len(name) - name[::-1].index("-") - 1
    method = name[:last_index]
    shift = name[last_index + 1:]

    mixture_results = {}
    with open(file, "r") as f:
        lines = f.readlines()
    for line in lines:
        if any([metric in line for metric in metrics]):
            metric = line.split(":")[0]
            value = float(line.split(":")[1])
            if metric not in mixture_results:
                mixture_results[metric] = value
                values[method][metric][shift] = value
            else:
                diffs[method][metric][shift] = mixture_results[metric] - value

for method in diffs:
    print(method)
    for metric in diffs[method]:
        print(metric)
        print("".join([f"({shift},{round(diffs[method][metric][shift] * 100, 2)})" for shift in sorted(diffs[method][metric].keys())]))
        print(", ".join([f"{shift}: {round(values[method][metric][shift] * 100, 2)}" for shift in sorted(values[method][metric].keys())]))
    print()

## Stats

In [ ]:
sums = defaultdict(int)

for file in glob(stats_directory + "/*.out"):
    with open(file, "r") as f:
        lines = f.readlines()
    for line in lines:
        if line.startswith("Output Type") or line.strip() == "":
            continue

        key, rest = line.split(":")
        value = int(rest.split("=")[0])

        sums[key] += value

for key, value in sums.items():
    print(f"{key}: {value}")

print()
print(f'% Unsure: {sums["Unsure Outputs"] / sums["Total Outputs"] * 100}')
print(f'% Non-Unsure: {sums["Non-Unsure Outputs"] / sums["Total Outputs"] * 100}')
print(f'% Unsure Correct: {sums["Unsure Correct"] / sums["Unsure Outputs"] * 100}')
print(f'% Non-Unsure Correct: {sums["Non-Unsure Correct"] / sums["Non-Unsure Outputs"] * 100}')
print(f'% Non-Unsure Shifted: {sums["Non-Unsure Shifted Outputs"] / sums["Total Shifted Outputs"] * 100}')
print(f'% Non-Unsure Shifted Correct: {sums["Non-Unsure Shifted Correct"] / sums["Non-Unsure Shifted Outputs"] * 100}')